<a href="https://colab.research.google.com/github/marybello/sms_text_classifier/blob/master/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
 
print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
tsv_read = pd.read_csv(train_file_path, sep='\t')
with open('train.csv','w') as write_csv:
    write_csv.write(tsv_read.to_csv(sep=',', index=False))
tsv_read2 = pd.read_csv(test_file_path, sep='\t')
with open('test.csv','w') as write_csv:
    write_csv.write(tsv_read2.to_csv(sep=',', index=False))    

In [ ]:
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
train_label = train_dataset['ham'].map({'ham': 0, 'spam': 1})
test_label = test_dataset['ham'].map({'ham': 0, 'spam': 1})
X_train = train_dataset.iloc[:,1]
X_test =  test_dataset.iloc[:,1]
print(len(train_label))
print(len(X_train))

In [ ]:
# Defining pre-processing hyperparameters
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)


In [ ]:
# Get the word_index 
word_index = tokenizer.word_index
tot_words = len(word_index)
tot_words

In [ ]:
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

In [ ]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

In [ ]:
vocab_size = 500 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 24

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
#Dense model architecture
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
# fitting a dense spam detector model
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, train_label, epochs=num_epochs, validation_data=(testing_padded, test_label),callbacks =[early_stop], verbose=2)

In [ ]:
model.evaluate(testing_padded, test_label)

In [ ]:
#LSTM hyperparameters
n_lstm = 20
drop_lstm =0.2

In [ ]:
#LSTM Spam detection architecture
model1 = Sequential()
model1.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(training_padded, train_label, epochs=num_epochs, validation_data=(testing_padded, test_label),callbacks =[early_stop], verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

from keras.preprocessing.sequence import pad_sequences
def predict_message(pred_text):
    sequence =  tokenizer.texts_to_sequences(pred_text)
    sequence = pad_sequences(sequence, max_len, padding = padding_type, truncating = trunc_type)
    # get the prediction
    prediction = model1.predict(sequence)
    #one-hot encoded vector, revert using np.argmax
    #return prediction
    if prediction[0].max() < 0.5:
      predicted = "ham"
    else:
      predicted = "spam" 

    return [prediction[0].max(), predicted]
   
  

pred_text = ["i'll bring it tomorrow. don't forget the milk."]

prediction = predict_message(pred_text)
prediction


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = [["how are you doing today"],
                   ["sale today! to stop texts call 98912460324"],
                   ["i dont want to go. can we try it a different day? available sat"],
                   ["our new mobile video service is live. just install on your phone to start watching."],
                   ["you have won £1000 cash! call to claim your prize."],
                   ["i'll bring it tomorrow. don't forget the milk."],
                   ["wow, is your arm alright. that happened to me one time too"]
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
